In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
os.chdir("/kaggle/input/youtube-comments-dataset")

In [2]:
os.getcwd()

'/kaggle/input/youtube-comments-dataset'

In [3]:
import pandas as pd
df=pd.read_csv("YoutubeCommentsDataSet.csv")

In [4]:
df

,Comment,Sentiment
0,lets not forget that apple pay in 2014 require...,neutral
1,here in nz 50 of retailers don’t even have con...,negative
2,i will forever acknowledge this channel with t...,positive
3,whenever i go to a place that doesn’t take app...,negative
4,apple pay is so convenient secure and easy to ...,positive
...,...,...
18403,i really like the point about engineering tool...,positive
18404,i’ve just started exploring this field and thi...,positive
18405,excelente video con una pregunta filosófica pr...,neutral
18406,hey daniel just discovered your channel a coup...,positive


In [5]:
df.isnull().sum()

Comment      44
Sentiment     0
dtype: int64

In [6]:
df.describe()

,Comment,Sentiment
count,18364,18408
unique,17871,3
top,one of the best thing about dude is that he ne...,positive
freq,10,11432


In [7]:
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
df["Sentiment"]=lb.fit_transform(df["Sentiment"])

In [8]:
df

,Comment,Sentiment
0,lets not forget that apple pay in 2014 require...,1
1,here in nz 50 of retailers don’t even have con...,0
2,i will forever acknowledge this channel with t...,2
3,whenever i go to a place that doesn’t take app...,0
4,apple pay is so convenient secure and easy to ...,2
...,...,...
18403,i really like the point about engineering tool...,2
18404,i’ve just started exploring this field and thi...,2
18405,excelente video con una pregunta filosófica pr...,1
18406,hey daniel just discovered your channel a coup...,2


In [9]:
df.isnull().sum()

Comment      44
Sentiment     0
dtype: int64

In [10]:
df["Comment"] = df["Comment"].fillna("No comment available")  # Replace NaN with text

In [11]:
df

,Comment,Sentiment
0,lets not forget that apple pay in 2014 require...,1
1,here in nz 50 of retailers don’t even have con...,0
2,i will forever acknowledge this channel with t...,2
3,whenever i go to a place that doesn’t take app...,0
4,apple pay is so convenient secure and easy to ...,2
...,...,...
18403,i really like the point about engineering tool...,2
18404,i’ve just started exploring this field and thi...,2
18405,excelente video con una pregunta filosófica pr...,1
18406,hey daniel just discovered your channel a coup...,2


In [12]:
df.isnull().sum()

Comment      0
Sentiment    0
dtype: int64

In [14]:
from sklearn.model_selection import train_test_split

In [22]:
x_train,x_test,y_train,y_test=train_test_split(df["Comment"],df["Sentiment"],random_state=42, test_size=0.2)

In [23]:
x_train

12791    subscribe or ill kick you also dont forget tha...
3097     hey thats a really great tutorial 5 minutes is...
16982           thanks for all the great work you folks do
7708     2323 la relajación es importante 18kissxuno lo...
7144     literature review definition a critical recap ...
                               ...                        
11284    education is used to separate and maintain the...
11964    i respect every one who were involved in this ...
5390     this is such an emotional experience to know w...
860      can we just take a moment to appreciate how mu...
15795    yeah but if you buy a rechargeable battery pac...
Name: Comment, Length: 14726, dtype: object

In [25]:
x_train[1]

'here in nz 50 of retailers don’t even have contactless credit card machines like paywave which support apple pay they don’t like the high fees that come with these'

In [26]:
from transformers import TFAutoModel, AutoTokenizer
import tensorflow as tf

model = TFAutoModel.from_pretrained("bert-base-cased")  


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [33]:
df["Sentiment"].nunique()

3

In [28]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [35]:
# Tokenize training and testing data with max_length
train_encodings = tokenizer(
    x_train.tolist(), padding=True, truncation=True, max_length=128, return_tensors="tf"
)
test_encodings = tokenizer(
    x_test.tolist(), padding=True, truncation=True, max_length=128, return_tensors="tf"
)

# Function to convert tokenized data to TensorFlow tensors
def convert_to_tf(tokenized_data, labels):
    input_ids = tf.convert_to_tensor(tokenized_data["input_ids"])
    attention_mask = tf.convert_to_tensor(tokenized_data["attention_mask"])
    labels = tf.convert_to_tensor(labels, dtype=tf.int32)  # Convert labels to int32
    
    return ({"input_ids": input_ids, "attention_mask": attention_mask}, labels)

# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices(convert_to_tf(train_encodings, y_train.values))
test_dataset = tf.data.Dataset.from_tensor_slices(convert_to_tf(test_encodings, y_test.values))

# Batch the datasets
BATCH_SIZE = 32
train_dataset = train_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

# Print one batch to check
for batch in train_dataset.take(1):
    print(batch)


({'input_ids': <tf.Tensor: shape=(32, 128), dtype=int32, numpy=
array([[  101,  4841,  1116, ...,     0,     0,     0],
       [  101, 23998,  1115, ...,     0,     0,     0],
       [  101,  5438,  1111, ...,     0,     0,     0],
       ...,
       [  101,  1142,  1108, ...,     0,     0,     0],
       [  101,  1115,  2298, ...,     0,     0,     0],
       [  101,   178,  1567, ...,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(32, 128), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}, <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 0, 2, 2, 1, 2, 1, 2,
       2, 2, 1, 2, 0, 2, 1, 2, 2, 2], dtype=int32)>)


In [36]:
class BERTForClassification(tf.keras.Model):
    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs):
        outputs = self.bert(**inputs)  # Unpack dictionary inputs correctly
        cls_token_output = outputs.last_hidden_state[:, 0, :]  #  Take the [CLS] token
        return self.fc(cls_token_output)


# Using outputs.last_hidden_state[:, 0, :] 
# ensures you always get the [CLS] token representation,
# which is best for classification tasks.

In [37]:
classifier= BERTForClassification(model,num_classes=3)
classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [40]:
history = classifier.fit(
    train_dataset,
    epochs=10,
    batch_size=8,  # Reduce batch size
    verbose=1
)


Epoch 1/10
461/461 [==============================] - 216s 469ms/step - loss: 0.4377 - accuracy: 0.8116
Epoch 2/10
461/461 [==============================] - 216s 468ms/step - loss: 0.2783 - accuracy: 0.8843
Epoch 3/10
461/461 [==============================] - 216s 468ms/step - loss: 0.1854 - accuracy: 0.9272
Epoch 4/10
461/461 [==============================] - 216s 468ms/step - loss: 0.1308 - accuracy: 0.9488
Epoch 5/10
461/461 [==============================] - 216s 468ms/step - loss: 0.0974 - accuracy: 0.9635
Epoch 6/10
461/461 [==============================] - 216s 469ms/step - loss: 0.0642 - accuracy: 0.9760
Epoch 7/10
461/461 [==============================] - 216s 469ms/step - loss: 0.0535 - accuracy: 0.9813
Epoch 8/10
461/461 [==============================] - 216s 469ms/step - loss: 0.0361 - accuracy: 0.9872
Epoch 9/10
461/461 [==============================] - 216s 468ms/step - loss: 0.0309 - accuracy: 0.9893
Epoch 10/10
461/461 [==============================] - 216s 468m

In [41]:
classifier.evaluate(test_dataset)

116/116 [==============================] - 22s 168ms/step - loss: 0.7943 - accuracy: 0.8419


[0.7943165898323059, 0.8419337272644043]

In [ ]:
# # Tokenize with max_length
# train_encodings = tokenizer(x_train.tolist(), padding=True, truncation=True, max_length=128)
# test_encodings = tokenizer(x_test.tolist(), padding=True, truncation=True, max_length=128)

# # Convert data to TensorFlow format
# train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train.values)).batch(32)
# test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test.values)).batch(32)

# # Print one batch
# for batch in train_dataset.take(1):
#     print(batch)


In [44]:
def predict_sentiment(text, model, tokenizer):
    # Tokenize input text
    encoding = tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors="tf")

    # Get model prediction
    prediction = classifier.predict(dict(encoding))

    # Get class with highest probability
    predicted_label = tf.argmax(prediction, axis=1).numpy()[0]

    # Label mapping
    labels = {0: "NEGATIVE", 1: "NEUTRAL", 2: "POSITIVE"}
    
    return labels[predicted_label]

# Example usage
text = "This song is amazing!"
result = predict_sentiment(text, model, tokenizer)
print(f"Sentiment: {result}")


1/1 [==============================] - 3s 3s/step
Sentiment: POSITIVE


In [45]:
def predict_sentiment(text, model, tokenizer):
    # Tokenize input text
    encoding = tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors="tf")

    # Get model prediction
    prediction = classifier.predict(dict(encoding))

    # Get class with highest probability
    predicted_label = tf.argmax(prediction, axis=1).numpy()[0]

    # Label mapping
    labels = {0: "NEGATIVE", 1: "NEUTRAL", 2: "POSITIVE"}
    
    return labels[predicted_label]

# Example usage
text =input("Enter your text:- ")
result = predict_sentiment(text, model, tokenizer)
print(f"Sentiment: {result}")


Enter your text:-  The video is worst 


1/1 [==============================] - 3s 3s/step
Sentiment: NEGATIVE
